In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta

In [2]:
chromedriver = "C:\Program Files\Google\Chrome\Application/chromedriver.exe" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [4]:
url_list = []
date_list = []
day_list = []
list_of_loc =  ["FRA-RUH","VIE-RUH","ORY-RUH",
               "GVA-RUH","AMS-RUH","BUD-RUH",
               "BRU-RUH","LHR-RUH","MAD-RUH",
               "PRG-RUH","LIS-RUH",]
def make_flight_url(starting_date):

    base_url1 = 'https://www.almosafer.com/en/flights/'
    base_url2 = '/Business/1Adult/'
    for location in list_of_loc:
        for i in range(0,7):
            starting_date_2 = (starting_date + timedelta(days=i))
            day_list.append(starting_date_2.strftime("%A"))
            date_for_url = str(starting_date_2.year) + "-" + str(starting_date_2.month).zfill(2) + "-" + str(starting_date_2.day).zfill(2)
            date_list.append(date_for_url)
            search_url = (base_url1 + location + "/" + date_for_url + base_url2)
            url_list.append(search_url)
    return url_list, day_list , date_list

starting_date = '12/01/21'
starting_date = datetime.strptime(starting_date, '%m/%d/%y')
make_flight_url(starting_date)

(['https://www.almosafer.com/en/flights/FRA-RUH/2021-12-01/Business/1Adult/',
  'https://www.almosafer.com/en/flights/FRA-RUH/2021-12-02/Business/1Adult/',
  'https://www.almosafer.com/en/flights/FRA-RUH/2021-12-03/Business/1Adult/',
  'https://www.almosafer.com/en/flights/FRA-RUH/2021-12-04/Business/1Adult/',
  'https://www.almosafer.com/en/flights/FRA-RUH/2021-12-05/Business/1Adult/',
  'https://www.almosafer.com/en/flights/FRA-RUH/2021-12-06/Business/1Adult/',
  'https://www.almosafer.com/en/flights/FRA-RUH/2021-12-07/Business/1Adult/',
  'https://www.almosafer.com/en/flights/VIE-RUH/2021-12-01/Business/1Adult/',
  'https://www.almosafer.com/en/flights/VIE-RUH/2021-12-02/Business/1Adult/',
  'https://www.almosafer.com/en/flights/VIE-RUH/2021-12-03/Business/1Adult/',
  'https://www.almosafer.com/en/flights/VIE-RUH/2021-12-04/Business/1Adult/',
  'https://www.almosafer.com/en/flights/VIE-RUH/2021-12-05/Business/1Adult/',
  'https://www.almosafer.com/en/flights/VIE-RUH/2021-12-06/Busin

In [5]:
columns = ["Flight_Number","Ariline","Class","Date","Day","Time_of_Landing","Time_of_arrival","Number_of_Stops","Duration","Baggage","Location","Destination","Price"]
flight_df = pd.DataFrame(columns=columns)

i = 0

for url in url_list[:]:
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    time.sleep(2)
    
    price_list =[]
    airline_list = []
    flight_num_list = []
    time_landing_list = []
    time_arrival_list = []
    Duration_list = []
    Baggage_list =[]
    stops_list =[]
    

    for j in range(8):
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1)

    soup = BeautifulSoup(driver.page_source)
    soup

    for rest in soup.find_all('div', class_='sc-cSYcjD hXnhxm')[:]: 
        price_list.append(rest.text)

    for rest in soup.find_all('div', class_='sc-dEfkYy dunONY')[:]: 
        airline_list.append(rest.find('div').text)

    for rest in soup.find_all('div', class_='sc-dEfkYy dunONY')[:]: 
        flight_num_list.append(rest.find_all('div')[-1].text)

    for rest in soup.find_all('div', class_='sc-bIqbHp aPzHZ sc-kpOJdX kAzjOl col-2')[:]: 
        time_landing_list.append(rest.find("div").text)

    for rest in soup.find_all('div', class_='sc-bIqbHp gwjyCQ sc-kpOJdX kAzjOl col-2')[:]: 
        time_arrival_list.append(rest.find("div").text)

    for rest in soup.find_all('div',class_="sc-jxGEyO dfxDXk sc-kpOJdX kAzjOl col-5")[:]:
        stops_list.append(rest.find("span").text)
        
    for rest in soup.find_all('span', class_='sc-FQuPU kdpQLw')[:]: 
        Duration_list.append(rest.text)

    for rest in soup.find_all('span', class_='sc-iuDHTM cnrMzL')[:]: 
        Baggage_list.append(rest.text)

    for M in range(len(flight_num_list)):
        flight_df.loc[len(flight_df)] = {"Flight_Number":flight_num_list[M],
                             "Ariline":airline_list[M],
                             "Class":"Business",
                             "Date":date_list[i],
                             "Day":day_list[i],
                             "Time_of_Landing":time_landing_list[M],
                             "Time_of_arrival":time_arrival_list[M],
                             "Number_of_Stops":stops_list[M],   
                             "Duration":Duration_list[M],
                             "Baggage":Baggage_list[M],
                             "Location":soup.find('div', class_='sc-yZwTr bmGDAz').text,
                             "Destination":"RUH",
                             "Price":price_list[M]}
    
    i += 1
    
    driver.close()
flight_df.to_csv("Business_FLIGHTS")

<ipython-input-5-1dc895020458>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


In [7]:
flight_df

,Flight_Number,Ariline,Class,Date,Day,Time_of_Landing,Time_of_arrival,Number_of_Stops,Duration,Baggage,Location,Destination,Price
0,RJ-126,Royal Jordanian,Business,2021-12-01,Wednesday,02:45pm,02:10am,1 Stop,09h 25m,Baggage included (40KG),FRA,RUH,"SAR4,970"
1,MS-9000,Egyptair,Business,2021-12-01,Wednesday,01:30pm,03:40am,1 Stop,12h 10m,Baggage included (40KG),FRA,RUH,"SAR3,927"
2,BA-903,British Airways,Business,2021-12-01,Wednesday,11:25am,11:40pm,1 Stop,10h 15m,Baggage included (40KG),FRA,RUH,"SAR6,258"
3,BA-8762,British Airways,Business,2021-12-01,Wednesday,10:15am,11:40pm,1 Stop,11h 25m,Baggage included (40KG),FRA,RUH,"SAR6,307"
4,TK-1594,Turkish Airlines,Business,2021-12-01,Wednesday,06:10pm,05:45am,1 Stop,09h 35m,Baggage included (40KG),FRA,RUH,"SAR7,363"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,TP-1352,Turkish Airlines,Business,2021-12-07,Tuesday,09:45am,05:45am,2 Stops,17h 00m,Baggage included (40KG),LIS,RUH,"SAR11,207"
1519,EY-1554,Etihad Airways,Business,2021-12-07,Tuesday,09:45am,11:05am,2 Stops,22h 20m,Baggage included (40KG),LIS,RUH,"SAR10,868"
1520,AF-1125,Air France,Business,2021-12-07,Tuesday,05:30am,07:15pm,1 Stop,10h 45m,Baggage included (40KG),LIS,RUH,"SAR12,587"
1521,ET-1730,Ethiopian Airlines,Business,2021-12-07,Tuesday,01:20pm,01:55am,2 Stops,33h 35m,Baggage included (60KG),LIS,RUH,"SAR11,789"


In [8]:
flight_df.to_csv("Business_FLIGHTS")

In [ ]:
# del flight_df